# 2.4.2 Evaluating Hyperparameters RNN
## This script contain the following points:
### 01. Importing Libraries and Data
### 02. Optimizing Hyperparameters
### 03. Running CNN Model with Optimized Parameters

### 01. Importing Libraria and Data

In [53]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [3]:
# Creating path variable
path = r'/home/cwidner/Documents/CareerFoundry/MachineLearning'

In [5]:
# Loading in cleaned weather dataset
df_weather = pd.read_csv(os.path.join(path,'02 Data','Prepared Data','weather_cleaned.csv'),index_col=False)
df_weather.shape

(22950, 136)

In [7]:
df_weather.drop(columns=['Unnamed: 0'], inplace=True)
df_weather.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [9]:
# Loading in cleaned pleasant dataset
df_pleasant = pd.read_csv(os.path.join(path,'02 Data','Prepared Data','pleasant_weather_cleaned.csv'),index_col=False)
df_pleasant.shape

(22950, 16)

In [11]:
df_pleasant.drop(columns=['Unnamed: 0'],inplace=True)
df_pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 02 Optimizing Hyperparameters

In [13]:
# Create matrices
X = df_weather
y = df_pleasant

# Check shapes
print(X.shape)
print(y.shape)

(22950, 135)
(22950, 15)


In [15]:
# Create matrices
X = X.to_numpy()
print(X.shape)

(22950, 135)


In [17]:
# Reshape X to (22950, 15, 9)

X = X.reshape(-1, 15, 9)
print(X.shape)

(22950, 15, 9)


In [19]:
y = np.array(y)

In [21]:
# Reshape y to (22950,)
y = np.argmax(y, axis=1)
print("y shape after argmax:", y.shape)

y shape after argmax: (22950,)


In [23]:
# Check unique values in yto ensure it's correct
print("Unique values in y:", np.unique(y))

Unique values in y: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]


In [25]:
# Check shape
y.shape

(22950,)

In [27]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=127)

In [29]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [31]:
# Determine the number of time steps for the input data
timesteps = X_train.shape[1]

# Determine the dimensionality of the input data
input_dim = X_train.shape[2]

# Specify the number of classes for the target variable
n_classes = 15  

# Create a scorer for accuracy
score_acc = make_scorer(accuracy_score)

In [33]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer_name = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def cnn_model():
        model = Sequential()
        model.add(Input(shape=(timesteps, input_dim)))
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=127))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))  # sigmoid softmax
        
        # Create a new optimizer instance for each iteration
        if optimizer_name == 'Adam':
            optimizer_instance = Adam(learning_rate=learning_rate)
        elif optimizer_name == 'SGD':
            optimizer_instance = SGD(learning_rate=learning_rate)
        elif optimizer_name == 'RMSprop':
            optimizer_instance = RMSprop(learning_rate=learning_rate)
        elif optimizer_name == 'Adadelta':
            optimizer_instance = Adadelta(learning_rate=learning_rate)
        elif optimizer_name == 'Adagrad':
            optimizer_instance = Adagrad(learning_rate=learning_rate)
        elif optimizer_name == 'Adamax':
            optimizer_instance = Adamax(learning_rate=learning_rate)
        elif optimizer_name == 'Nadam':
            optimizer_instance = Nadam(learning_rate=learning_rate)
        elif optimizer_name == 'Ftrl':
            optimizer_instance = Ftrl(learning_rate=learning_rate)
        
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer_instance, metrics=['accuracy'])
        return model

    # K-fold cross-validation
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=20)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=127)
    results = []
    for train, test in kfold.split(X, y):
        model = cnn_model()
        model.fit(X[train], y[train], epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])
        scores = model.evaluate(X[test], y[test], verbose=1)
        results.append(scores[1])  # Assuming accuracy is the second metric
    return np.mean(results)


In [35]:
# Start timing the Bayesian Optimization process
start = time.time()

# Define the hyperparameter space for Bayesian Optimization
params = {
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation': (0, 9),  # 9
    'optimizer': (0, 7),  # 7
    'learning_rate': (0.001, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs': (20, 100),
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0.3, 0.5)
}

# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=127)
nn_opt.maximize(init_points=15, n_iter=4)  # 25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 27: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6451 - loss: 147.1270
Epoch 25: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6552 - loss: 39.1203
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6520 - loss: 44.6849
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6617 - loss: 55.7117
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0306 - loss: 96.9454  
| 1         | 0.5218    | 4.714     | 232.0     | 0.186     | 0.4546    | 64.17     | 1.173     | 1.883     | 2.431     | 0.6709    | 52.56     | 0.9062

In [37]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

Best Parameters: {'activation': 3.238346333286747, 'batch_size': 858.0863899431488, 'dropout': 0.40228837505867143, 'dropout_rate': 0.393459528897498, 'epochs': 60.44669476125228, 'kernel': 2.1120927218648693, 'layers1': 2.1169196169026234, 'layers2': 2.3365351568358195, 'learning_rate': 0.06152500093002812, 'neurons': 12.581466622198223, 'normalization': 0.45787219367534804, 'optimizer': 0.902427852988836}
Highest Accuracy: 0.8191721081733704


In [39]:
# Retrieve the best parameters from the optimization result
optimum = nn_opt.max['params']

# Assign the best parameters to their respective variables
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU', 'relu']
activation = activationL[round(optimum['activation'])]

# Convert the hyperparameters to their integer form where necessary
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])
optimum['kernel'] = round(optimum['kernel'])

optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
optimizerD = {
    'SGD': SGD(learning_rate=learning_rate),
    'Adam': Adam(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}

# Retrieve the optimizer name
optimizer_name = optimizerL[round(optimum['optimizer'])]

# Print the optimum parameters in a readable format
print(f"Best Parameters: ")
print(f"Activation: {activation}")
print(f"Batch Size: {optimum['batch_size']}")
print(f"Dropout Rate: {optimum['dropout_rate']:.4f}")
print(f"Epochs: {optimum['epochs']}")
print(f"Kernel Size: {optimum['kernel']}")
print(f"Layers1: {optimum['layers1']}")
print(f"Layers2: {optimum['layers2']}")
print(f"Learning Rate: {optimum['learning_rate']:.4f}")
print(f"Neurons: {optimum['neurons']}")
print(f"Normalization: {optimum['normalization']:.4f}")
print(f"Optimizer: {optimizer_name}")

Best Parameters: 
Activation: softsign
Batch Size: 858
Dropout Rate: 0.3935
Epochs: 60
Kernel Size: 2
Layers1: 2
Layers2: 2
Learning Rate: 0.0615
Neurons: 13
Normalization: 0.4579
Optimizer: Adam


### 03. Running CNN Model with Optimized Parameters

In [41]:
# Best parameters from optimization
best_params = {
    'neurons': 13,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adam',
    'learning_rate': 0.0615,
    'batch_size': 858,
    'epochs':60,
    'layers1': 2,
    'layers2': 2,
    'normalization': 0.4579,
    'dropout': 0.40228837505867143,
    'dropout_rate':  0.3935
}

# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(15,9)))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate'], seed=123))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # sigmoid, tanh, softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


Epoch 1/60


/home/cwidner/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 1s - 40ms/step - accuracy: 0.6045 - loss: 1.2757
Epoch 2/60
21/21 - 0s - 5ms/step - accuracy: 0.6394 - loss: 1.0503
Epoch 3/60
21/21 - 0s - 5ms/step - accuracy: 0.6550 - loss: 0.9599
Epoch 4/60
21/21 - 0s - 5ms/step - accuracy: 0.6692 - loss: 0.9179
Epoch 5/60
21/21 - 0s - 5ms/step - accuracy: 0.6868 - loss: 0.8643
Epoch 6/60
21/21 - 0s - 5ms/step - accuracy: 0.6995 - loss: 0.8259
Epoch 7/60
21/21 - 0s - 5ms/step - accuracy: 0.7121 - loss: 0.8089
Epoch 8/60
21/21 - 0s - 5ms/step - accuracy: 0.7364 - loss: 0.7434
Epoch 9/60
21/21 - 0s - 5ms/step - accuracy: 0.7350 - loss: 0.7318
Epoch 10/60
21/21 - 0s - 5ms/step - accuracy: 0.7510 - loss: 0.6970
Epoch 11/60
21/21 - 0s - 5ms/step - accuracy: 0.7508 - loss: 0.7247
Epoch 12/60
21/21 - 0s - 5ms/step - accuracy: 0.7655 - loss: 0.6681
Epoch 13/60
21/21 - 0s - 5ms/step - accuracy: 0.7628 - loss: 0.6837
Epoch 14/60
21/21 - 0s - 5ms/step - accuracy: 0.7838 - loss: 0.6129
Epoch 15/60
21/21 - 0s - 5ms/step - accuracy: 0.7613 - loss: 0.6800

In [43]:
model.summary()

Model: "sequential_95"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_95 (Conv1D)              │ (None, 14, 13)         │           247 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_470 (Dense)               │ (None, 14, 13)         │           182 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_471 (Dense)               │ (None, 14, 13)         │           182 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_472 (Dense)               │ (None, 14, 13)         │           182 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_473 (Dense)               │ (None, 14, 13)         │           182 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_95 (MaxPooling1D) │ (None, 7, 13)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_95 (Flatten)            │ (None, 91)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_474 (Dense)               │ (None, 15)             │         1,380 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,067 (27.61 KB)

 Trainable params: 2,355 (9.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,712 (18.41 KB)

In [45]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
# Assuming X_train and y_train are your input data and labels
# One-hot encode y_train
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [69]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [73]:
model.fit(X_train, y_train_one_hot, batch_size=858, epochs=60, verbose=2)

Epoch 1/60
21/21 - 1s - 40ms/step - accuracy: 0.8379 - loss: 0.4395
Epoch 2/60
21/21 - 0s - 5ms/step - accuracy: 0.8471 - loss: 0.4056
Epoch 3/60
21/21 - 0s - 5ms/step - accuracy: 0.8529 - loss: 0.3965
Epoch 4/60
21/21 - 0s - 5ms/step - accuracy: 0.8537 - loss: 0.3917
Epoch 5/60
21/21 - 0s - 5ms/step - accuracy: 0.8545 - loss: 0.3896
Epoch 6/60
21/21 - 0s - 5ms/step - accuracy: 0.8545 - loss: 0.3877
Epoch 7/60
21/21 - 0s - 5ms/step - accuracy: 0.8549 - loss: 0.3869
Epoch 8/60
21/21 - 0s - 5ms/step - accuracy: 0.8563 - loss: 0.3874
Epoch 9/60
21/21 - 0s - 5ms/step - accuracy: 0.8568 - loss: 0.3845
Epoch 10/60
21/21 - 0s - 5ms/step - accuracy: 0.8571 - loss: 0.3830
Epoch 11/60
21/21 - 0s - 5ms/step - accuracy: 0.8563 - loss: 0.3831
Epoch 12/60
21/21 - 0s - 5ms/step - accuracy: 0.8573 - loss: 0.3823
Epoch 13/60
21/21 - 0s - 5ms/step - accuracy: 0.8576 - loss: 0.3814
Epoch 14/60
21/21 - 0s - 5ms/step - accuracy: 0.8577 - loss: 0.3810
Epoch 15/60
21/21 - 0s - 5ms/step - accuracy: 0.8589 - l

In [75]:
# Creating stations dictionary
stations = {
    0: 'BASEL',
    1: 'BELGRADE',
    2: 'BUDAPEST',
    3: 'DEBILT',
    4: 'DUSSELDORF',
    5: 'HEATHROW',
    6: 'KASSEL',
    7: 'LJUBJANA',
    8: 'MAASTRICHT',
    9: 'MADRID',
    10: 'MUNCHENB',
    11: 'OSLO',
    12: 'SONNBLICK',
    13: 'STOCKHOLM',
    14: 'VALENTIA'
}

In [93]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])



In [97]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

In [99]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step


In [101]:
# Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 84.87%


In [103]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [105]:
# Convert y_test_one_hot back to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

In [107]:
cm = confusion_matrix(y_test, y_pred, stations)
print(cm)

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3505       130        11       4           6        10       0   
BELGRADE      233       845        14       2           0         0       0   
BUDAPEST       56        62        82       1           2         9       0   
DEBILT         29         9         9      28           0         4       0   
DUSSELDORF      8         6         4       6           6         1       0   
HEATHROW       13        12         7       3           2        44       0   
KASSEL          2         7         0       3           0         0       3   
LJUBJANA       10         6         3       0           0         3       0   
MAASTRICHT      0         0         0       0           1         1       0   
MADRID         17        39         9       1           0        11       0   
MUNCHENB        6         3         2       0       